In [30]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json
import requests
from bs4 import BeautifulSoup
import os
import shutil

In [18]:
producer = KafkaProducer(bootstrap_servers=['3.27.226.3:9092'], #change ip here
                         value_serializer=lambda x:
                         dumps(x).encode('utf-8'))

In [28]:
def ProducerPlus(stock_code, start_page, end_page): # 初始化变量i
    while True:
        all_comments = []  # 初始化存储评论的列表
        for page_num in range(start_page, end_page + 1):
            url = f"http://guba.eastmoney.com/list,{stock_code},f_{page_num}.html"
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(url, headers=headers)
    
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                comments = soup.find_all('tr', class_='listitem')  # 获取所有评论项
                for comment in comments:
                    # 提取所需字段
                    read = comment.find('div', class_='read').text if comment.find('div', class_='read') else 'N/A'
                    reply = comment.find('div', class_='reply').text if comment.find('div', class_='reply') else 'N/A'
                    title_div = comment.find('div', class_='title')
                    title = title_div.get_text(strip=True) if title_div else 'N/A'
                    author = comment.find('div', class_='author').text if comment.find('div', class_='author') else 'N/A'
                    update_time = comment.find('div', class_='update').text if comment.find('div',
                                                                                            class_='update') else 'N/A'
    
                    # 收集数据
                    all_comments.append({
                        "页码": page_num,
                        "阅读": read,
                        "回复": reply,
                        "标题": title,
                        "作者": author,
                        "更新时间": update_time
                    })
            else:
                print(f"请求失败，状态码：{response.status_code}")
    # 将数据转换为DataFrame
        df = pd.DataFrame(all_comments)
        # 数据处理完成后，进行文件保存和差异计算等操作
        fileNameNew = f"comments_{stock_code}New.csv"
        fileNameOld = f"comments_{stock_code}Old.csv"
        df.to_csv(fileNameNew, index=False, encoding='utf_8_sig')
        print(f"数据已保存到CSV文件：{fileNameNew}")
        if not os.path.isfile(fileNameOld):
            print('creating old file')
            with open(fileNameOld, "w") as fold:
                fold.write("页码,阅读,回复,标题,作者,更新时间")
        # 计算New和Old文件的差异
        df_old=pd.read_csv(fileNameOld)
        difference = pd.merge(df, df_old, on='标题', how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
        dict_difference = difference.to_dict(orient='records')
        print(difference)
        for row_dict in fileNameNew:
            producer.send('Stock', value=dict_difference)
            sleep(1)
        shutil.copyfile(fileNameNew, fileNameOld)
        print('moving new file to old file')
        sleep(100)
  # 使用os.replace代替os.system和mv命令

In [29]:
ProducerPlus('300750', 1, 2)

数据已保存到CSV文件：comments_300750New.csv
   页码_x 阅读_x 回复_x                             标题          作者_x       更新时间_x  \
4     1    6    0                         牛，还有翻倍          百股花香  04-01 08:38   
5     1    5    0                        宁德时代能好吗  股友2l6737337a  04-01 08:38   
6     1   29    0  2024.4.1A股分析（应更注意操作品种自身技术面信息）         白白胖胖0  04-01 08:37   
7     1   39    0                   满仓满融杀入，坐等起飞！         带散户斗庄  04-01 08:26   

   页码_y  阅读_y  回复_y 作者_y 更新时间_y  
4   NaN   NaN   NaN  NaN    NaN  
5   NaN   NaN   NaN  NaN    NaN  
6   NaN   NaN   NaN  NaN    NaN  
7   NaN   NaN   NaN  NaN    NaN  


KeyboardInterrupt: 

In [24]:
producer.flush()